In [1]:
import itertools
import pandas as pd
import numpy as np
import toytree
import toyplot
import arviz as az
import pymc3 as pm

### Tree and dataframe setup

In [2]:
# generate a random tree
NSPECIES = 80
TREE = toytree.rtree.bdtree(
    ntips=NSPECIES,
    seed=666,
).mod.node_scale_root_height(1.0)

# node idxs that delimit several distinct clades on this tree
CLADES = [152, 153, 154, 155]

# draw and color the four major clades
TREE.draw(
    layout='d', 
    width=500,
    tip_labels=False,
    edge_colors=TREE.get_edge_values_mapped({
        j: toytree.colors[i] for i, j in enumerate(CLADES)
    }),
    scalebar=True,
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t9cbc545b643b4dc8a21ff1deb750b24d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0.0 0.5 1.0

In [3]:
# make group index (gidx)
crown_dict = {i: TREE.get_tip_labels(i) for i in CLADES}
gidx = np.zeros(TREE.ntips, dtype=int)
for tidx, tip in enumerate(TREE.get_tip_labels()):
    for cidx, clade in enumerate(crown_dict):
        if tip in crown_dict[clade]:
            gidx[tidx] = cidx
gidx

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [4]:
# True param values
𝛼_mean = 0.05
𝛼_std = 0.02
𝛽_mean = 3.0
𝛽_std = 0.2
𝜓_mean = 0.0
𝜓_std = 0.33

# 4 different clade effects on rate of RI (used for partial-pooling data)
𝜓_0_mean = 1.0
𝜓_0_std = 0.1
𝜓_1_mean = 0.5
𝜓_1_std = 0.05
𝜓_2_mean = -0.5
𝜓_2_std = 0.05
𝜓_3_mean = -1.0
𝜓_3_std = 0.1

In [51]:
# species dataframe
SPECIES_DATA = pd.DataFrame({
    "gidx": gidx,
    "b": np.random.normal(𝛽_mean, 𝛽_std, TREE.ntips),
    "psi": np.random.normal(𝜓_mean, 𝜓_std, TREE.ntips),
    "psi_x": np.concatenate([
        np.random.normal(𝜓_0_mean, 𝜓_0_std, len(gidx[gidx == 0])),
        np.random.normal(𝜓_1_mean, 𝜓_1_std, len(gidx[gidx == 1])),
        np.random.normal(𝜓_2_mean, 𝜓_2_std, len(gidx[gidx == 2])),
        np.random.normal(𝜓_3_mean, 𝜓_3_std, len(gidx[gidx == 3])),
    ]),
})
SPECIES_DATA.head()

,gidx,b,psi,psi_x
0,0,2.998961,0.365273,1.079141
1,0,2.936681,-0.547848,0.957262
2,0,2.802832,-0.150277,1.040836
3,0,2.960643,0.360127,0.844379
4,0,3.186135,0.206865,0.987505


### Generate crossing data

In [53]:
def get_dist(tree, idx0, idx1):
    "returns the genetic distance between two nodes on a tree"
    dist = tree.treenode.get_distance(
        tree.idx_dict[idx0], 
        tree.idx_dict[idx1],
    )
    return dist

# get all combinations of two sampled taxa
a, b = zip(*itertools.combinations(range(NSPECIES), 2))

# organize into DF and get genetic distance between pairs
DATA = pd.DataFrame({
    "sidx0": a,
    "sidx1": b,
    "dist": [(get_dist(TREE, i, j) / 2) for (i, j) in zip(a, b)],
})

DATA['b'] = np.random.normal(𝛽_mean, 𝛽_std, DATA.shape[0])
DATA['velo'] = (
    DATA['b']
    + SPECIES_DATA['psi'][DATA.sidx0].values
    + SPECIES_DATA['psi'][DATA.sidx1].values
)
DATA['velo_x'] = (
    DATA['b']
    + SPECIES_DATA['psi_x'][DATA.sidx0].values
    + SPECIES_DATA['psi_x'][DATA.sidx1].values
)
DATA['intercept'] = np.random.normal(𝛼_mean, 𝛼_std, DATA.shape[0])
# DATA['error'] = np.random.normal(0.0, 𝜎_std, DATA.shape[0])

# get logits
DATA['logit_b'] = (
    1 / (1 + np.exp(-(DATA.intercept + DATA.b * np.log(DATA.dist))))
)
DATA['logit'] = (
    1 / (1 + np.exp(-(DATA.intercept + DATA.velo * np.log(DATA.dist))))
)
DATA['logit_x'] = (
    1 / (1 + np.exp(-(DATA.intercept + DATA.velo_x * np.log(DATA.dist))))
)

# get RI estimates
DATA['RI_pooled'] = np.random.binomial(n=1, p=DATA.logit_b / DATA.logit_b.max())
DATA['RI_unpooled'] = np.random.binomial(n=1, p=DATA.logit / DATA.logit.max())
DATA['RI_partpooled'] = np.random.binomial(n=1, p=DATA.logit_x / DATA.logit_x.max())

DATA.head()

,sidx0,sidx1,dist,b,velo,velo_x,intercept,logit_b,logit,logit_x,RI_pooled,RI_unpooled,RI_partpooled
0,0,1,0.073376,3.290771,3.108196,5.327174,0.053578,0.000195,0.000314,9.547395e-07,0,0,0
1,0,2,0.089748,3.140516,3.355512,5.260493,0.037017,0.000534,0.000318,3.224566e-06,0,0,0
2,0,3,0.129770,2.916170,3.641570,4.839690,0.044254,0.002703,0.000616,5.336296e-05,0,0,0
3,0,4,0.144542,2.815045,3.387183,4.881691,0.041696,0.004482,0.001487,8.268546e-05,0,0,0
4,0,5,0.144542,2.852265,3.599622,4.928176,0.061829,0.004257,0.001006,7.711301e-05,0,0,0


In [54]:
NSAMPLES = 1000
SAMPLE = DATA.sample(NSAMPLES).copy().reset_index(drop=True)
SAMPLE.head()

,sidx0,sidx1,dist,b,velo,velo_x,intercept,logit_b,logit,logit_x,RI_pooled,RI_unpooled,RI_partpooled
0,8,62,1.000000,3.522723,3.955839,3.940824,0.019467,0.504867,0.504867,0.504867,1,1,1
1,36,47,1.000000,3.070545,2.748627,3.098635,0.059017,0.514750,0.514750,0.514750,1,1,0
2,35,43,0.742448,2.934870,3.327025,4.537597,0.033362,0.301398,0.277392,0.211160,1,1,1
3,9,54,1.000000,3.092278,1.829145,3.817094,0.050671,0.512665,0.512665,0.512665,1,1,1
4,15,55,1.000000,3.035191,3.646696,3.494298,0.087729,0.521918,0.521918,0.521918,0,1,1


### Visualize data

In [18]:
def logit_plot(dist, logit, RI):
    canvas = toyplot.Canvas(width=500, height=250)
    ax0 = canvas.cartesian(
        label="pooled data (function)",
        xlabel="Genetic dist.",
        ylabel="Logit function",
        grid=(1, 2, 0),
    )
    ax1 = canvas.cartesian(
        label="pooled data (observation)",
        xlabel="Genetic dist.",
        ylabel="RI",
        grid=(1, 2, 1),
    )

    # points are jittered on x-axis for visibility
    ax0.scatterplot(
        dist,
        logit,
        size=5,
        opacity=0.33,
        color=toyplot.color.Palette()[0],
    );
    ax1.scatterplot(
        dist,
        RI,
        size=10,
        opacity=0.2,
        marker="|",
        mstyle={
            "stroke": toyplot.color.Palette()[1],
            "stroke-width": 3,
        },
    );
    return canvas, (ax0, ax1)

In [55]:
logit_plot(SAMPLE.dist, SAMPLE.logit_b, SAMPLE.RI_pooled);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tae02eb8b4f3a43388f50f342fe18fd80" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0" transform=

In [56]:
logit_plot(SAMPLE.dist, SAMPLE.logit, SAMPLE.RI_unpooled);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t72a4c821eac847ef8c551aa91a00b016" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0" transform="translat

In [57]:
logit_plot(SAMPLE.dist, SAMPLE.logit_x, SAMPLE.RI_partpooled);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="te88ed03a736748de8f8252c64e2f6d67" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0" transform="translate(199.99999999999994, 72.0

### Define models

In [58]:
def pooled_logistic(x, y, **kwargs):
    
    # define model
    with pm.Model() as model:  

        # parameters and error
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        
        # link function
        effect = 𝛼 + 𝛽 * np.log(x)
        logit = pm.Deterministic("logit", pm.invlogit(effect))
        
        # data likelihood
        y = pm.Bernoulli("y", p=logit, observed=y)
        
        # sample posterior, skip burnin
        trace = pm.sample(**kwargs)[1000:]
    
        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

In [59]:
def unpooled_logistic(x, y, idx0, idx1, **kwargs):
    
    # define model
    with pm.Model() as model:
        
        # indexers
        sidx0 = pm.Data("spp_idx0", idx0.values)
        sidx1 = pm.Data("spp_idx1", idx1.values)

        # parameters and error
        𝜓_mean = pm.Normal('𝜓_mean', mu=0., sigma=5., shape=1)
        𝜓_std = pm.HalfNormal('𝜓_std', 5., shape=1)
        𝜓_offset = pm.Normal('𝜓_offset', mu=0, sigma=1., shape=TREE.ntips)
        𝜓 = pm.Deterministic('𝜓', 𝜓_mean + 𝜓_std * 𝜓_offset)
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        
        # link function
        effect = 𝛼 + (𝛽 + 𝜓[sidx0] + 𝜓[sidx1]) * np.log(x)
        logit = pm.Deterministic("logit", pm.invlogit(effect))
        
        # data likelihood
        y = pm.Bernoulli("y", p=logit, observed=y)
        
        # sample posterior, skip burnin
        trace = pm.sample(init = 'adapt_diag', **kwargs)[1000:]
    
        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

In [60]:
def partpooled_logistic(x, y, idx0, idx1, gidx, **kwargs):
    
    # define model
    with pm.Model() as model:
        
        # indexers
        sidx0 = pm.Data("spp_idx0", idx0)
        sidx1 = pm.Data("spp_idx1", idx1)
        gidx = pm.Data("gidx", gidx)

        # parameters and error
        𝜓_mean = pm.Normal('𝜓_mean', mu=0., sigma=5., shape=4)
        𝜓_std = pm.HalfNormal('𝜓_std', 5., shape=4)
        𝜓_offset = pm.Normal('𝜓_offset', mu=0, sigma=1., shape=TREE.ntips)
        𝜓 = pm.Deterministic('𝜓', 𝜓_mean[gidx] + 𝜓_std[gidx] * 𝜓_offset)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        
        # linear model prediction
        effect = 𝛼 + (𝛽 + 𝜓[sidx0] + 𝜓[sidx1]) * np.log(x)
        logit = pm.Deterministic("logit", pm.invlogit(effect))
        
        # data likelihood (normal distributed errors)
        y = pm.Bernoulli("y", p=logit, observed=y)

        # sample posterior, skip burnin
        trace = pm.sample(init = 'adapt_diag', **kwargs)[1000:]

        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

### Functions to plot results

In [25]:
def toytrace(trace, var_names, titles):
    """
    Plot posterior trace with toyplot
    """
    nvars = len(var_names)
    
    # setup canvase
    canvas = toyplot.Canvas(width=500, height=200 * nvars)
    
    # store axes
    axes = []
    
    # iter over params
    for pidx, param in enumerate(var_names):
        
        # get param posterior
        posterior = trace.get_values(param)
        
        # setup axes 
        ax = canvas.cartesian(grid=(nvars, 1, pidx))
        ax.y.show = False
        ax.x.spine.style = {"stroke-width": 1.5}
        ax.x.ticks.labels.style = {"font-size": "12px"}
        ax.x.ticks.show = True
        ax.x.label.text = f"param='{titles[pidx]}'"        
        
        # iterate over shape of param
        for idx in range(posterior.shape[1]):
            mags, bins = np.histogram(posterior[:, idx], bins=100)
            ax.plot(bins[1:], mags, stroke_width=2, opacity=0.6)
        axes.append(ax)
    return canvas, axes

In [26]:
import scipy.stats as stats

def draw_velocity_dists(trace, baseline=0.15):
    """
    Draw the clade velocities as gaussians
    """
    canvas = toyplot.Canvas(width=350, height=300)
    axes = canvas.cartesian(xlabel="Relative velocity of reproductive isolation")
    marks = []
    base = 0
    for i in range(trace['𝜓_mean'].shape[1]):
        
        loc = trace['𝜓_mean'][:, i].mean()
        scale = trace['𝜓_std'][:, i].mean()
        interval = stats.norm.interval(0.995, loc, scale)
        points = np.linspace(interval[0], interval[1], 100)
        mark = axes.fill(
            points, 
            stats.norm.pdf(points, loc=loc, scale=scale), 
            style={
                "fill-opacity": 0.45,
                "stroke": 'black',
                "stroke-opacity": 1.0,
                "stroke-width": 1,
            },
            baseline=np.repeat(base, 100),
        )
        marks.append(mark)
        axes.hlines(base, style={"stroke-dasharray": "5,5", 'stroke-width': 1})
        base += baseline
        
    axes.y.show = False
    axes.x.ticks.locator = toyplot.locator.Extended(only_inside=True)
    axes.x.ticks.show = True
    return canvas, axes, marks

In [61]:
# MCMC sampler kwargs
sample_kwargs = dict(
    tune=10000,
    draws=10000,
    target_accept=0.99,
    return_inferencedata=False,
    progressbar=True,
)

### Run three datasets under pooled model

In [62]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# pooled model
pooled_model_pooled_data = pooled_logistic(*model_args[:2], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 194 seconds.


In [63]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# pooled model
pooled_model_unpooled_data = pooled_logistic(*model_args[:2], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 190 seconds.


In [64]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# pooled model
pooled_model_partpooled_data = pooled_logistic(*model_args[:2], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 201 seconds.


### Run three datasets under unpooled model

In [65]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
unpooled_model_pooled_data = unpooled_logistic(*model_args[:4], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 8480 seconds.


In [66]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
unpooled_model_unpooled_data = unpooled_logistic(*model_args[:4], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 9506 seconds.


In [ ]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
unpooled_model_partpooled_data = unpooled_logistic(*model_args[:4], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼, 𝜓_offset, 𝜓_std, 𝜓_mean]


### Run three datasets under partpooled model

In [73]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
partpooled_model_pooled_data = partpooled_logistic(*model_args, **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 3960 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [74]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
partpooled_model_unpooled_data = partpooled_logistic(*model_args, **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 3824 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [75]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
partpooled_model_partpooled_data = partpooled_logistic(*model_args, **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 4000 seconds.


In [76]:
partpooled_model_partpooled_data['stats']

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
𝜓_mean[0],2.536,2.332,-2.019,6.740,0.021,0.015,12428.0,12428.0,12434.0,19191.0,1.0
𝜓_mean[1],2.494,2.385,-2.009,6.977,0.021,0.015,12737.0,12737.0,12733.0,19914.0,1.0
𝜓_mean[2],-0.962,2.272,-5.258,3.268,0.021,0.015,12001.0,12001.0,12003.0,18209.0,1.0
𝜓_mean[3],-1.072,2.295,-5.270,3.349,0.021,0.015,12169.0,12169.0,12172.0,18197.0,1.0
𝜓_offset[0],0.012,0.942,-1.760,1.785,0.003,0.006,84133.0,12774.0,84259.0,25160.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
logit[995],0.002,0.005,0.000,0.006,0.000,0.000,33140.0,33140.0,35555.0,31562.0,1.0
logit[996],0.003,0.013,0.000,0.010,0.000,0.000,32506.0,32506.0,47737.0,30822.0,1.0
logit[997],0.954,0.009,0.937,0.970,0.000,0.000,62841.0,62841.0,63406.0,25586.0,1.0
logit[998],0.954,0.009,0.937,0.970,0.000,0.000,62841.0,62841.0,63406.0,25586.0,1.0


In [77]:
toytrace(partpooled_model_partpooled_data['trace'], ['𝜓_mean', '𝜓_offset', '𝜓'], ['psi-mean', 'psi-offset', 'psi-spp']);

<svg class="toyplot-canvas-Canvas" height="600.0px" id="t67235f57c6bb4b8f82dda77af35232d2" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 600.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 110.41761770188512 149.9244142101285 L 113.84774277560346 150.0 L 117.27786784932178 150.0 L 120.70799292304012 150.0 L 124.13811799675847 150.0 L 127.5682430704768 149.84882842025698 L 130.9983681441951 150.0 L 134.42849321791346 149.9244142101285 L 137.85861829163179 149.9244142101285 L 141.28874336535011 149.697656840514 L 144.71886843906844 149.77324263038548 L 148.14899351278677 149.77324263038548 L 151.5791185865051 149.31972789115648 L 155.00924366022343 149.6220710506425 L 158.43936873394176 149.697656840514 L 161.8694938076601 148.86621315192747 L 165.29961888137845 148.56386999244143 L 168.72974395509675 148.48828420256993 L 172.1598690288151 148.0347694633409 L 175.5899941025334 147.65684051398338 L 179.02011917625177 147.12773998488285 L 182.45024424997013 147.05215419501135 L 185.88036932368843 144.8601662887377 L 189.31049439740676 144.02872260015116 L 192.74061947112506 144.10430839002268 L 196.17074454484342 141.23204837490553 L 199.60086961856175 139.56916099773244 L 203.0309946922801 138.43537414965985 L 206.4611197659984 133.06878306878306 L 209.89124483971673 133.37112622826908 L 213.32136991343506 131.25472411186698 L 216.75149498715342 122.41118669690098 L 220.18162006087172 122.63794406651549 L 223.61174513459008 119.31216931216932 L 227.04187020830838 113.64323507180649 L 230.47199528202677 109.25925925925925 L 233.90212035574507 102.75888133030989 L 237.33224542946337 99.35752078609221 L 240.76237050318173 93.46182917611489 L 244.19249557690006 91.34542705971278 L 247.6226206506184 82.27513227513228 L 251.05274572433672 75.92592592592592 L 254.48287079805505 68.66969009826153 L 257.9129958717734 72.37339380196524 L 261.3431209454917 60.430839002267575 L 264.77324601921003 62.8495842781557 L 268.20337109292836 57.860922146636426 L 271.6334961666467 58.163265306122454 L 275.063621240365 54.15721844293273 L 278.4937463140834 56.80272108843537 L 281.9238713878017 57.936507936507944 L 285.35399646152007 58.541194255479965 L 288.7841215352384 52.26757369614512 L 292.21424660895667 55.82010582010582 L 295.644371682675 62.16931216931217 L 299.0744967563934 62.62282690854119 L 302.50462183011166 67.00680272108843 L 305.93474690383005 71.46636432350718 L 309.3648719775484 77.13529856386998 L 312.79499705126665 84.31594860166288 L 316.22512212498503 84.99622071050642 L 319.65524719870336 92.93272864701436 L 323.0853722724217 97.31670445956159 L 326.51549734614 104.87528344671202 L 329.9456224198583 107.06727135298564 L 333.3757474935767 113.26530612244899 L 336.80587256729507 115.15495086923659 L 340.23599764101334 121.80650037792894 L 343.66612271473167 123.31821617535903 L 347.09624778845 124.8299319727891 L 350.52637286216833 129.8185941043084 L 353.9564979358867 135.63869992441423 L 357.386623009605 135.71428571428572 L 360.8167480833233 138.7377173091459 L 364.24687315704165 139.4935752078609 L 367.67699823076 140.854119425548 L 371.1071233044783 142.6681783824641 L 374.53724837819664 144.25547996976567 L 377.967373451915 146.2962962962963 L 381.39749852563335 146.0695389266818 L 384.8276235993516 147.80801209372638 L 388.25774867306995 148.11035525321242 L 391.6878737467884 148.86621315192747 L 395.11799882050667 147.80801209372638 L 398.548123894225 149.24414210128495 L 401.9782489679433 148.48828420256993 L 405.40837404166155 149.24414210128495 L 408.83849911538 149.546485260771 L 412.2686241890983 149.697656840514 L 415.6987492628166 149.546485260771 L 419.

In [78]:
# show plot of TRUE vs. ESTIMATED rates
c, a, m = toyplot.scatterplot(
    partpooled_model_partpooled_data['trace']['𝜓'].mean(axis=0),         # estimated
    SPECIES_DATA['psi_x'],                             # true
    width=400,
    height=250,
    xlabel="ESTIMATED species velocity",
    ylabel="TRUE species velocity",
    color=[toyplot.color.Palette()[i] for i in SPECIES_DATA.gidx],
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t0528947f877b49f389d046fb78f96523" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 250.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> -2 0 2 ESTIMATED species velocity -1 0 1 TRUE species velocity

In [79]:
draw_velocity_dists(partpooled_model_partpooled_data['trace'], baseline = 1.2);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t41609b6a0cb340cd8133315c0697a50c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <polygon points="156.43046471840253,250.0 157.8806620444793,250.0 159.330859370556,250.0 160.7810566966328,250.0 162.2312540227095,250.0 163.68145134878625,250.0 165.13164867486302,250.0 166.58184600093975,250.0 168.03204332701648,250.0 169.4822406530932,250.0 170.93243797916995,250.0 172.3826353052467,250.0 173.83283263132344,250.0 175.28302995740017,250.0 176.73322728347694,250.0 178.1834246095537,250.0 179.6336219356304,250.0 181.08381926170716,250.0 182.5340165877839,250.0 183.98421391386063,250.0 185.4344112399374,250.0 186.88460856601412,250.0 188.33480589209088,250.0 189.7850032181676,250.0 191.23520054424435,250.0 192.6853978703211,250.0 194.13559519639784,250.0 195.58579252247455,250.0 197.0359898485513,250.0 198.48618717462807,250.0 199.9363845007048,250.0 201.38658182678154,250.0 202.8367791528583,250.0 204.28697647893503,250.0 205.7371738050118,250.0 207.1873711310885,250.0 208.63756845716526,250.0 210.087765783242,250.0 211.53796310931875,250.0 212.98816043539549,250.0 214.43835776147222,250.0 215.88855508754898,250.0 217.3387524136257,250.0 218.78894973970245,250.0 220.2391470657792,250.0 221.68934439185594,250.0 223.13954171793264,250.0 224.5897390440094,250.0 226.03993637008617,250.0 227.49013369616293,250.0 228.94033102223963,250.0 230.39052834831637,250.0 231.84072567439313,250.0 233.2909230004699,250.0 234.74112032654665,250.0 236.19131765262335,250.0 237.6415149787001,250.0 239.09171230477685,250.0 240.5419096308536,250.0 241.99210695693031,250.0 243.44230428300705,250.0 244.8925016090838,250.0 246.34269893516054,250.0 247.7928962612373,250.0 249.24309358731404,250.0 250.6932909133908,250.0 252.1434882394675,250.0 253.59368556554426,250.0 255.043882891621,250.0 256.49408021769773,250.0 257.9442775437745,250.0 259.3944748698512,250.0 260.84467219592796,250.0 262.2948695220047,250.0 263.7450668480815,250.0 265.1952641741582,250.0 266.64546150023494,250.0 268.09565882631165,250.0 269.5458561523884,250.0 270.99605347846517,250.0 272.4462508045419,250.0 273.89644813061864,250.0 275.3466454566954,250.0 276.79684278277216,250.0 278.2470401088489,250.0 279.6972374349256,250.0 281.14743476100233,250.0 282.59763208707903,250.0 284.04782941315585,250.0 285.49802673923256,250.0 286.9482240653093,250.0 288.39842139138614,250.0 289.8486187174628,250.0 291.2988160435396,250.0 292.7490133696163,250.0 294.199210695693,250.0 295.6494080217698,250.0 297.09960534784653,250.0 298.54980267392324,250.0 300.0,250.0 300.0,249.6382776030396 298.54980267392324,249.5765439413475 297.09960534784653,249.50586603010174 295.6494080217698,249.42524272067993 294.199210695693,249.33361124682125 292.7490133696163,249.2298519035086 291.2988160435396,249.11279446392797 289.8486187174628,248.9812264781624 288.39842139138614,248.83390357444156 286.9482240653093,248.6695618540577 285.49802673923256,248.4869324343335 284.04782941315585,248.28475815037854 282.59763208707903,248.06181237612836 281.14743476100233,247.8169198689335 279.6972374349256,247.54897948065704 278.2470401088489,247.25698851303878 276.79684278277216,246.940068427482 275.3466454566954,246.59749155117748 273.89644813061864,246.22870835459895 272.4462508045419,245.83337481209597 270.99605347846517,245.4113792999138 269.5458561523884,244.96286843690274 268.09565882631165,244.4882712348486 266.64546150023494,243.98832090006 265.1952641741582,243.46407361769616 263.7450668480815,242.91692365711805 262.2948695220047,242.3486141617154 260.84467219592

### Assess model fit

In [43]:
def rmse(predictions, targets):
    differences = predictions - targets                       #the DIFFERENCEs.
    differences_squared = differences ** 2                    #the SQUAREs of ^
    mean_of_differences_squared = differences_squared.mean()  #the MEAN of ^
    rmse_val = np.sqrt(mean_of_differences_squared)           #ROOT of ^
    return rmse_val   

In [44]:
def aicm(mean, variance):
    return 2*mean - 2*variance

In [45]:
rmse(unpooled_sub['trace']['𝜓'].mean(axis=0), SPECIES_DATA['psi'])

1.6829399085369496

In [46]:
rmse(partpooled_sub['trace']['𝜓'].mean(axis=0), SPECIES_DATA['psi_x'])

1.2960935050421183

In [47]:
aicm(unpooled_sub['trace']['𝜓'].mean(), unpooled_sub['trace']['𝜓'].var())

-24.07642765287162

In [48]:
# Raftery et al. (2007) holds that this value is better.  R+M disagree?
aicm(partpooled_sub['trace']['𝜓'].mean(), partpooled_sub['trace']['𝜓'].var())

-17.436591387294683

In [80]:
az_pooled_model_pooled_data = az.from_pymc3(trace = pooled_model_pooled_data['trace'], 
                                            model = pooled_model_pooled_data['model'])
az_pooled_model_unpooled_data = az.from_pymc3(trace = pooled_model_unpooled_data['trace'], 
                                            model = pooled_model_unpooled_data['model'])
az_pooled_model_partpooled_data = az.from_pymc3(trace = pooled_model_partpooled_data['trace'], 
                                            model = pooled_model_partpooled_data['model'])
az_unpooled_model_pooled_data = az.from_pymc3(trace = unpooled_model_pooled_data['trace'], 
                                            model = unpooled_model_pooled_data['model'])
az_unpooled_model_unpooled_data = az.from_pymc3(trace = unpooled_model_unpooled_data['trace'], 
                                            model = unpooled_model_unpooled_data['model'])
az_unpooled_model_partpooled_data = az.from_pymc3(trace = unpooled_model_partpooled_data['trace'], 
                                            model = unpooled_model_partpooled_data['model'])
az_partpooled_model_pooled_data = az.from_pymc3(trace = partpooled_model_pooled_data['trace'], 
                                            model = partpooled_model_pooled_data['model'])
az_partpooled_model_unpooled_data = az.from_pymc3(trace = partpooled_model_unpooled_data['trace'], 
                                            model = partpooled_model_unpooled_data['model'])
az_partpooled_model_partpooled_data = az.from_pymc3(trace = partpooled_model_partpooled_data['trace'], 
                                            model = partpooled_model_partpooled_data['model'])

In [81]:
az.compare({"pooled_model_pooled_data": az_pooled_model_pooled_data,
            "pooled_model_unpooled_data": az_pooled_model_unpooled_data, 
            "pooled_model_partpooled_data": az_pooled_model_partpooled_data})

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:150: UserWarning: 
The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive accuracy.
  "\nThe scale is now log by default. Use 'scale' argument or "


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
pooled_model_pooled_data,0,-324.33,2.1962,0,0.90178,18.5939,0,False,log
pooled_model_unpooled_data,1,-358.191,2.30109,33.8604,0.0980357,18.8728,25.2834,False,log
pooled_model_partpooled_data,2,-390.841,2.82162,66.511,0.000184241,22.387,27.2619,False,log


In [82]:
az.compare({"unpooled_model_pooled_data": az_unpooled_model_pooled_data,
            "unpooled_model_unpooled_data": az_unpooled_model_unpooled_data, 
            "unpooled_model_partpooled_data": az_unpooled_model_partpooled_data})

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:150: UserWarning: 
The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive accuracy.
  "\nThe scale is now log by default. Use 'scale' argument or "
/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:684: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  "Estimated shape parameter of Pareto distribution is greater than 0.7 for "
/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:684: Use

,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
unpooled_model_pooled_data,0,-320.521,35.3211,0,0.617966,19.2584,0,True,log
unpooled_model_partpooled_data,1,-329.843,53.7655,9.32147,0.326374,20.2225,24.3726,True,log
unpooled_model_unpooled_data,2,-350.928,43.6695,30.4068,0.0556603,18.5592,25.3953,True,log


In [83]:
az.compare({"partpooled_model_pooled_data": az_partpooled_model_pooled_data,
            "partpooled_model_unpooled_data": az_partpooled_model_unpooled_data, 
            "partpooled_model_partpooled_data": az_partpooled_model_partpooled_data})

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:150: UserWarning: 
The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive accuracy.
  "\nThe scale is now log by default. Use 'scale' argument or "
/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:684: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  "Estimated shape parameter of Pareto distribution is greater than 0.7 for "
/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:684: Use

,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
partpooled_model_partpooled_data,0,-314.432,33.8041,0,0.582105,21.6858,0,True,log
partpooled_model_pooled_data,1,-320.324,34.4865,5.89155,0.401581,17.8145,24.7119,True,log
partpooled_model_unpooled_data,2,-356.786,51.575,42.354,0.0163138,18.8658,27.2717,True,log


In [84]:
az.compare({"pooled_model_pooled_data": az_pooled_model_pooled_data,
            "unpooled_model_unpooled_data": az_unpooled_model_unpooled_data, 
            "partpooled_model_partpooled_data": az_partpooled_model_partpooled_data})

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:150: UserWarning: 
The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive accuracy.
  "\nThe scale is now log by default. Use 'scale' argument or "
/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:684: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  "Estimated shape parameter of Pareto distribution is greater than 0.7 for "
/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:684: Use

,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
partpooled_model_partpooled_data,0,-314.432,33.8041,0,0.637551,19.7688,0,True,log
pooled_model_pooled_data,1,-324.33,2.1962,9.89803,0.333209,18.0489,24.7382,False,log
unpooled_model_unpooled_data,2,-350.928,43.6695,36.4957,0.0292403,18.5039,25.3989,True,log
